In [15]:
# constant input, single pre-learned pattern
# Single layer, input -> output -> input inh -> output inh -> input gain...
# Check if sinusoidal dynamics develop

In [62]:
import pdb
import sys
sys.path.append('/Users/amolk/work/AGI/pattern-machine/src')
%load_ext autoreload
%autoreload 1
from IPython.display import display
print = display

%aimport patternmachine.layer
%aimport patternmachine.signal_grid_set
%aimport patternmachine.signal_utils
%aimport patternmachine.trace
%aimport patternmachine.pattern_similarity
%aimport patternmachine.pattern_grid
%aimport patternmachine.clock
%aimport patternmachine.utils
%aimport patternmachine.input_output_patch_grid

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [63]:
%matplotlib inline
import matplotlib.pyplot as plt

def show_1d_image(image, title=None):
    assert len(image.shape) == 1
    fig=plt.figure(figsize=(2, 1))
    plt.axis('off')
#     ax = fig.add_subplot(111)
    if title is not None:
        plt.title(label=title)
    plt.imshow(image.unsqueeze(0), vmin=0, vmax=1)
#     ax.set_aspect('equal')
#     plt.tight_layout(pad=0)
    plt.show()
    
def show_2d_image(image, title=None):
    assert len(image.shape) == 2
    plt.axis('off')
    if title is not None:
        plt.title(label=title)
    plt.imshow(image)
    plt.show()

In [64]:
from typing import Tuple

import matplotlib.pyplot as plt
import torch
# from tqdm.notebook import tqdm
from tqdm import tqdm

from patternmachine.layer import Layer, LayerHP
from patternmachine.signal_grid import SignalGridHP, SignalGrid
from patternmachine.signal_grid_set import SignalGridSet
from patternmachine.signal_utils import SignalUtils
from patternmachine.input_output_patch_grid import InputOutputPatchGrid
from patternmachine.pattern_similarity import PatternSimilarity

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class DynamicsExperiment1:
    def __init__(self, mu_shape: Tuple = (1, 1), pattern_grid_shape: Tuple = (1, 1)):
        self.mu_shape = mu_shape
        self.pattern_grid_shape = pattern_grid_shape
        val = 0.3

        mu_signal_grid_hp = SignalGridHP(grid_shape = (1, 1), signal_shape = mu_shape)
        mu_pixels = SignalGrid(hp=mu_signal_grid_hp, init_pixel_value = val)
        mu_precision = SignalGrid(hp=mu_signal_grid_hp, init_pixel_value = 1.0)

        self.mu = SignalGridSet.from_signal_grids({"pixels": mu_pixels, "precision": mu_precision})
        print(self.mu.components["pixels"].pixels)
        print(self.mu.components["precision"].pixels)

        # Layer
        self.layer_hp = self.create_layer_hp(per_patch_pattern_grid_shape=pattern_grid_shape)
        self.layer = Layer(hp=self.layer_hp)

        self.layer.patterns.pixels_begin.components["mu"].pixels.data = torch.Tensor([[val]])
        self.layer.patterns.precision_begin.components["mu"].pixels.data = torch.Tensor([[1.0]])
        self.layer.patterns.pixels_end.components["mu"].pixels.data = torch.Tensor([[val]])
        self.layer.patterns.precision_end.components["mu"].pixels.data = torch.Tensor([[1.0]])

        self.layer.patterns.pixels_begin.components["__output__"].pixels.data = torch.Tensor([[1.0]])
        self.layer.patterns.precision_begin.components["__output__"].pixels.data = torch.Tensor([[1.0]])
        self.layer.patterns.pixels_end.components["__output__"].pixels.data = torch.Tensor([[1.0]])
        self.layer.patterns.precision_end.components["__output__"].pixels.data = torch.Tensor([[1.0]])

        print(self.layer.patterns.pixels)

        input_output_patch_grid = InputOutputPatchGrid(
            patch_grid_shape=self.layer.hp.patch_grid_shape,
            input_csg=self.mu,
            output=self.layer.output,
            output_patch_neighborhood_shape=self.layer.hp.output_patch_neighborhood_shape,
            per_patch_pattern_grid_shape=self.layer.hp.per_patch_pattern_grid_shape,
            input_coverage_factor=1.0,
        )
        print("input_output_patch_grid", input_output_patch_grid)

        pattern_similarity_begin = PatternSimilarity(
            signal=input_output_patch_grid.patches,
            patterns=self.layer.patterns.pixels_begin,
            patterns_precision=self.layer.patterns.precision_begin,
        )

    def create_layer_hp(
        self,
        input_coverage_factor=1.0,
        patch_grid_shape=(1, 1),
        per_patch_pattern_grid_shape=(1, 1),
        output_patch_neighborhood_shape=(1, 1),
        output_tau=1.0,
    ):
        hp = LayerHP(
            input_signal_shapes={"mu": self.mu_shape},
            input_coverage_factor=input_coverage_factor,
            patch_grid_shape=patch_grid_shape,
            per_patch_pattern_grid_shape=per_patch_pattern_grid_shape,
            output_patch_neighborhood_shape=output_patch_neighborhood_shape,
            output_decay=output_tau,
        )  # set output_decay=1.0 for IID data
        return hp

experiment = DynamicsExperiment1()

tensor([[0.3000]])

tensor([[1.]])

TypeError: __init__() got an unexpected keyword argument 'init_precision_value'

In [123]:
import torch
a = torch.ones(1,1) * 0.1
b = torch.ones(1,1) * 0.999

c = torch.logit(torch.clamp_min(a, min=0.01)) + torch.logit(torch.clamp_min(b, min=0.01))
torch.sigmoid(c)

tensor([[0.9911]])